In [1]:
import json
import random

from dotenv import load_dotenv
from google import genai
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [3]:
class FootballClubList(BaseModel):
    clubs: list[str] = Field(
        description="A list of exactly 10 major European football club names."
    )

In [4]:
client = genai.Client()

agent_sportscaster = Agent(
    name="sportscaster",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="You are a sportscaster.",
)
runner_sportscaster = InMemoryRunner(agent=agent_sportscaster)

agent_lister = Agent(
    name="lister",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="List 10 major European football clubs.",
    output_schema=FootballClubList,
)
runner_lister = InMemoryRunner(agent=agent_lister)


In [5]:
response_lister_raw = await runner_lister.run_debug(
    "List 10 major European football clubs"
)
response_lister = FootballClubList.model_validate_json(response_lister_raw[0].content.parts[0].text)
clubs =response_lister.clubs
print("clubs:", clubs)


 ### Created new session: debug_session_id

User > List 10 major European football clubs
lister > {
  "clubs": [
    "Real Madrid",
    "Barcelona",
    "Bayern Munich",
    "Manchester United",
    "Liverpool",
    "Juventus",
    "Paris Saint-Germain",
    "AC Milan",
    "Inter Milan",
    "Borussia Dortmund"
  ]
}
clubs: ['Real Madrid', 'Barcelona', 'Bayern Munich', 'Manchester United', 'Liverpool', 'Juventus', 'Paris Saint-Germain', 'AC Milan', 'Inter Milan', 'Borussia Dortmund']


In [6]:
print("Select a club:")
for i, club in enumerate(clubs, 1):
    print(f"{i}. {club}")

selection = int(input("Enter number: "))
human_club = clubs[selection - 1]
print(f"You selected: {human_club}")

Select a club:
1. Real Madrid
2. Barcelona
3. Bayern Munich
4. Manchester United
5. Liverpool
6. Juventus
7. Paris Saint-Germain
8. AC Milan
9. Inter Milan
10. Borussia Dortmund
You selected: Manchester United


In [7]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"What's the arch rival of {human_club}? Just give me the name of the club.",
)
computer_club = response.text
print(f"The arch rival of {human_club} is {computer_club}.")

The arch rival of Manchester United is Liverpool.


In [ ]:
agent_computer = Agent(
    name="computer",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=f"""
    You are the manager of the football club {computer_club}. You are responsible for the draft and the match between {human_club} and {computer_club}.
    Your goal is to assemble the strongest team possible and select an optimal formation to maximize your chances of winning against a human opponent.
    Context:
        * You are drafting from a dynamically generated pool of 16 players per round, for a total of 3 rounds.
        * Each player has one or more natural positions (Forward, Midfielder, Defender, Goalkeeper) and a star rating (1–6).
        * Star calculations:
            * Playing in one of their natural positions → full stars
            * Out-of-position → stars −1
            * Non-GK playing as GK → stars = 1
            * GK playing in another position → stars = 1
        * H (human) is your opponent. Drafts alternate. You cannot pick a player already chosen.
    Drafting Strategy:
        1. Prioritize selecting the strongest players in positions you need.
        2. Maintain a balanced squad:
            * Draft exactly 18 players.
            * Target positional distribution:
                2 Goalkeepers
                6 Defenders
                6 Midfielders
                4 Forwards
            * Players with multiple natural positions can be assigned strategically to maintain this balance.
            * Ensure that at the end of the draft, all positional requirements are met and the squad is valid for match day.
        3. Consider which positions your opponent is likely to pick and adjust your picks to counter them.
        4. Avoid overloading one position at the expense of squad balance.
    Formation Strategy:
        * Decide a formation for match day after drafting. There are only 5 valid formations: 4-4-2, 4-3-3, 3-5-2, 3-4-3 and 3-3-4.
        * Formation must be compatible with your drafted players. Multi-position players can be assigned to the position that maximizes stars or tactical advantage.
        * Optimize for midfield strength first, then attack and defense.
        * Ensure at least 1 goalkeeper is in the starting lineup.
    Decision Rules:
        * Always draft exactly 18 players by the end of 3 rounds.
        * When choosing substitutes or lineup adjustments, prioritize stars in natural positions.
        * Use tactical balance to cover weaknesses in your squad.
        * Multi-position players should be assigned to the position that gives the most star advantage for the lineup.
    """,
)
runner_computer = InMemoryRunner(agent=agent_computer)

agent_administrator = Agent(
    name="administrator",
    model=Gemini(
        # model="gemini-2.5-flash-lite",
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
        You are the administrator of the draft and the match between {human_club} and {computer_club}.
        You have the capability to create pools of real players with varying skill levels.
        These players can be from any team in the world. They can be from the past or present.
        Once a player is shown, he must not be shown again.
        Player positions and stars:
            * Natural positions: Forward, Midfielder, Defender, Goalkeeper
            * Stars: 1–6
            * A play can have one or multiple natural positions
        When a player plays in a match, the following rules apply in terms of starts:
            * Natural position → full stars
            * Out-of-position → stars −1
            * Non-GK playing GK → stars = 1
            * GK playing other → stars = 1
    """,
)
runner_administrator = InMemoryRunner(agent=agent_administrator)

In [9]:
response_sportscaster_raw = await runner_sportscaster.run_debug(
    f"There is a draft and a match between {human_club} and {computer_club}. Briefly introduce the history of both clubs."
)


 ### Created new session: debug_session_id

User > There is a draft and a match between Manchester United and Liverpool. Briefly introduce the history of both clubs.
sportscaster > Alright, folks, settle in! We're on the eve of a monumental clash, a fixture that sends shivers down the spine of football fans worldwide – Manchester United versus Liverpool! And with a draft looming, it's a perfect time to remember the sheer weight of history these two titans carry.

Let's start with **Manchester United**. The Red Devils, born out of a railway workers' club in 1878 as Newton Heath. They've seen it all – triumphs, tragedies, and an unparalleled era of dominance under Sir Matt Busby and later, Sir Alex Ferguson. Think Busby Babes, the Munich air disaster, the Treble in '99, and a trophy cabinet overflowing with league titles and European silverware. They're a global powerhouse, a symbol of resilience and relentless ambition.

And then, you have **Liverpool**. The Anfield club, founded way b

In [10]:
human_squad = []
computer_squad = []

In [ ]:
def human_choose_player(players):
    while True:
        player_choice = input("Choose a player (or stop to finish this round): ").strip()
        if player_choice in players:
            player_info = players[player_choice]
            print(f"You selected: {player_choice}")
            print(f"Player info: {player_info}")
            players.pop(player_choice)
            return {player_choice: player_info}
        elif player_choice == "stop":
            return None
        else:
            print("Invalid player. Please try again.")

# def computer_choose_player(players):
#     player_choice = random.choice(list(players.keys()))
#     print(f"Computer selected: {player_choice}")
#     player_info = players[player_choice]
#     print(f"Player info: {player_info}")
#     players.pop(player_choice)
#     return {player_choice: player_info}


In [12]:
def print_team_formation(players):
    """
    Prints players grouped by position in the order: 
    Forward, Midfielder, Defender, Goalkeeper.
    
    Args:
        players: List of dicts or objects with 'name', 'position', and 'stars'.
    """
    # 1. Define the explicit print order
    order = ["Forward", "Midfielder", "Defender", "Goalkeeper"]
    
    # 2. Group players by position
    # Use a dictionary to hold lists for each position
    squad = {pos: [] for pos in order}

    for player in players:
        for name, info in player.items():
            positions = info['positions']
            for pos in positions:
                if pos in squad:
                    squad[pos].append([name, info])
                    break
    
    # 3. Print strictly in the defined order
    for pos in order:
        if squad[pos]:  # Only print if there are players in this position
            print(f"{pos}s")
            print("-" * 20)
            for each in squad[pos]:
                print(each)
            print()

In [ ]:
async def draft_players():
    response_administrator_raw = await runner_administrator.run_debug(
        f"""
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        The star ratings should be distributed as evenly as possible across the pool.

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```". 
        """
    )
    response_administrator = response_administrator_raw[0].content.parts[0].text.replace("```json", "").replace("```", "")
    players = json.loads(response_administrator)
    for name, info in players.items():
        print(name, info)

    human_finished = False
    computer_finished = False

    while players and (not human_finished or not computer_finished):
        if len(human_squad) >= 18:
            human_finished = True
        if len(computer_squad) >= 18:
            computer_finished = True

        print()
        print("players to choose from:")
        for name, info in players.items():
            print(name, info)
        print()

        if not human_finished:
            human_choice = human_choose_player(players)
            if human_choice:
                human_squad.append(human_choice)
            else:
                human_finished = True

        if not computer_finished:
            computer_choice = computer_choose_player(runner_computer, players)
            if computer_choice:
                computer_squad.append(computer_choice)
            else:
                computer_finished = True

        print("human_squad size:", len(human_squad))
        print("computer_squad size:", len(computer_squad))
        print()

        print("human_squad:")
        print_team_formation(human_squad)
        print("computer_squad:")
        print_team_formation(computer_squad)
        print()


In [14]:
for i in range(3):
    await draft_players()


 ### Created new session: debug_session_id

User > 
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        The star ratings should be distributed as evenly as possible across the pool.

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```". 
        
administrator > ```json
{
    "Lionel Messi": {
        "positions": ["Forward"],
        "stars": 6
    },
    "Zinedine Zidane": {
        "positions": ["Midfielder"],
        "stars": 6
    },
    "Virgil van Dijk": {
        "positions": ["Defender"],
        "stars": 5
    },
    "Kevin De Bruy

In [15]:
human_squad

[{'Zinedine Zidane': {'positions': ['Midfielder'], 'stars': 6}},
 {'Lionel Messi': {'positions': ['Forward'], 'stars': 6}},
 {'Virgil van Dijk': {'positions': ['Defender'], 'stars': 5}},
 {'Alisson Becker': {'positions': ['Goalkeeper'], 'stars': 5}},
 {'Son Heung-min': {'positions': ['Forward', 'Midfielder'], 'stars': 4}},
 {'Declan Rice': {'positions': ['Midfielder'], 'stars': 4}},
 {'William Saliba': {'positions': ['Defender'], 'stars': 4}},
 {'Cristiano Ronaldo': {'positions': ['Forward'], 'stars': 6}},
 {'Paolo Maldini': {'positions': ['Defender'], 'stars': 6}},
 {'Luka Modrić': {'positions': ['Midfielder'], 'stars': 5}},
 {'Thierry Henry': {'positions': ['Forward'], 'stars': 5}},
 {'Manuel Neuer': {'positions': ['Goalkeeper'], 'stars': 5}},
 {"N'Golo Kanté": {'positions': ['Midfielder'], 'stars': 4}},
 {'Johan Cruyff': {'positions': ['Forward', 'Midfielder'], 'stars': 6}},
 {'Ronaldinho': {'positions': ['Midfielder', 'Forward'], 'stars': 6}},
 {'Mohamed Salah': {'positions': ['For

In [16]:
computer_squad

[{'Kevin De Bruyne': {'positions': ['Midfielder'], 'stars': 5}},
 {'Ollie Watkins': {'positions': ['Forward'], 'stars': 3}},
 {'Ben Mee': {'positions': ['Defender'], 'stars': 2}},
 {'Divock Origi': {'positions': ['Forward'], 'stars': 1}},
 {'Wout Weghorst': {'positions': ['Forward'], 'stars': 2}},
 {'Ben Foster': {'positions': ['Goalkeeper'], 'stars': 1}},
 {'Kieran Trippier': {'positions': ['Defender'], 'stars': 3}},
 {'James Ward-Prowse': {'positions': ['Midfielder'], 'stars': 3}},
 {'Scott McTominay': {'positions': ['Midfielder'], 'stars': 2}},
 {'Jordan Pickford': {'positions': ['Goalkeeper'], 'stars': 3}},
 {'Harry Kane': {'positions': ['Forward'], 'stars': 4}},
 {'Alexis Mac Allister': {'positions': ['Midfielder'], 'stars': 3}},
 {'Conor Gallagher': {'positions': ['Midfielder'], 'stars': 2}},
 {'Trent Alexander-Arnold': {'positions': ['Defender', 'Midfielder'],
   'stars': 4}},
 {'Harry Maguire': {'positions': ['Defender'], 'stars': 2}},
 {'Phil Jones': {'positions': ['Defender']

In [17]:
def get_squad_stars(squad):
    stars = 0
    # player is a dict: the key is the player name, the value is a dict with keys "positions" and "stars"
    for player in squad:
        for name, info in player.items():
            stars += info['stars']
    return stars

human_squad_stars = get_squad_stars(human_squad)
computer_squad_stars = get_squad_stars(computer_squad)

print("human_squad_stars:", human_squad_stars)
print("computer_squad_stars:", computer_squad_stars)

human_squad_stars: 92
computer_squad_stars: 44


In [ ]:
if len(human_squad) != 18 or 